In [59]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from numba import jit

In [60]:
# Define parameters
beta = 0.99 # Discount rate
r = 0.05 # Return on A
a = 1 # Productivity in production of K
delta = 0.05 # Depreciation rate of K
alpha = 0.5 # Weight on leisure
gamma = 0.5 # Weight on bequest
sigma = 0.5 # Unused
params = (beta, r, a, delta, alpha, gamma, sigma)

In [61]:
# Define g(x) and g'(x) (labor market equilibrium tradeoff) function
@jit
def g(x):
    k = 5/4
    s = 1/2
    d = 1/(k**0.5 - s)
    y = k - (x/d + s)**2
    
    if y <= 0:
        return 0
    
    return y

In [62]:
# Define g(x) and g'(x) (labor market equilibrium tradeoff) function
@jit
def g(x):
    k = 5/4
    s = 1/2
    a = 1/(k**0.5 - s)
    y = k - (x/a + s)**2
    
    return y

@jit
def g_prime(x):
    k = 5/4
    s = 1/2
    a = 1/(k**0.5 - s)
    y = (-2*(x/a + s))/a
    
    return y

In [63]:
# Define utility functions
@jit
def u(c, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if c <= 0:
        return -1e999
    
    return (c**(1-alpha))/(1-alpha)

@jit
def u_prime(c, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if c <= 0:
        return 1e999
    
    return c**(-alpha)

@jit
def v(l, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if l <= 0:
        return -1e999
    
    return (l**(1-gamma))/(1-gamma)

@jit
def v_prime(l, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if l <= 0:
        return 1e999
    
    return l**(-gamma)

@jit
def B(A, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if A <= 0:
        return -1e999
    
    return 100*(A**(1-sigma))/(1-sigma)

@jit
def B_prime(A, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if A <= 0:
        return 1e999
    
    return 100*A**(-sigma)

In [64]:
# Define a function that returns utility given c, l, and x
@jit
def utility(values, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    c, l, x = values # Unpack values to evaluate at
    
    u = - np.log(c) - alpha*np.log(l)
    return u

In [65]:
# Define constraint function
@jit
def constraints(values, states, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    A0, K0, A1, K1 = states # Unpack chosen states
    c, l, x = values # Unpack values to evaluate at
    
    err1 = (1+r)*A0 + g(x)*(1-l)*K0 - A1 - c
    err2 = (1 + a*x*(1-l) - delta)*K0 - K1
    err3 = l
    err4 = x
    err5 = c
    
    return np.array([err1, err2, err3, err4, err5])

In [66]:
A0 = 1
A1 = 1.01
K0 = 1
K1 = 1.01
states = (A0, K0, A1, K1)

In [67]:
# Define constraints
cons = {'type': 'ineq',
        'fun': constraints,
        'args': (states, params),
        'lb': np.array([0, 0, 0, 0, 0]),
        'ub': np.array([0, 0, 1, 1, np.inf]),
       }

In [68]:
results = minimize(utility, [0.5, 0.5, 0.5], method = 'SLSQP', constraints = cons, args = (params,))

In [69]:
# Define function that checks feasibility of chosen A_t+1 and K_t+1, assuming c = 0 and l = 0
@jit
def feasible(states, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    A0, K0, A1, K1 = states # Unpack given state evolution
    
    # Assuming limit case of h = 1, human capital evolution pins down x, which in turn pins down c
    x = (1/a)*(K1/K0 - 1 + delta)
    c = (1+r)*A0 + g(x)*K0 - A1
    
    # If c is positive, then allocation is feasible
    if c > 0:
        return True
    else:
        return False

In [70]:
check = feasible(states, params)

In [71]:
# Define a function that solves FOCs given state evolution
@jit
def foc(guess, states, params):
    c, l, x, mu, bl1, bx0, bx1 = guess # Unpack guess
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    A0, K0, A1, K1 = states # Unpack given state evolution
    
    foc1 = alpha/l + (1/c)*(-g(x))*K0 + mu*(-a*x*K0) - bl1
    foc2 = (1/c)*g_prime(x)*(1-l)*K0 + mu*a*(1-l)*K0 + bx0 - bx1
    foc3 = (1+r)*A0 + g(x)*(1-l)*K0 - A1 - c
    foc4 = (1 + a*x*(1-l) - delta)*K0 - K1
    
    # Complementary slackness
    comp1 = bl1*(1-l)
    comp2 = bx0*x
    comp3 = bx1*(1-x)
    
    return np.array([foc1, foc2, foc3, foc4, comp1, comp2, comp3])

In [72]:
# Want to define a function that returns flow utility of given a state evolution choice
@jit
def flow_choice(states, params):
    beta, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    A0, K0, A1, K1 = states # Unpack given state evolution
    
    # If unfeasible, return large negative utility
    check = feasible(states, params)
    if check == False:
        return -1e999
    
    # If feasible, return optimal choice of c, l, x
    c, l, x, mu, bl1, bx0, bx1 = fsolve(foc, [0.5, 0.5, 0.5, 1, 1, 1, 1], args = (states, params))
    
    return c, l, x, mu, bl1, bx0, bx1

In [73]:
# Set up state grids
A_grid_min = 1
A_grid_max = 5
A_grid_size = 10
K_grid_min = 1
K_grid_max = 5
K_grid_size = 10
T = 10
K_grid = np.linspace(K_grid_min, K_grid_max, K_grid_size)
A_grid = np.linspace(A_grid_min, A_grid_max, A_grid_size)
T_grid = np.linspace(0, T, T+1, dtype = int)

# Set up array to store value function
V_init = np.ones((A_grid_size, K_grid_size, T+1))
V_next = np.ones((A_grid_size, K_grid_size, T+1))

# Set up array to store state policy function
A_policy_grid = np.zeros((A_grid_size, K_grid_size, T+1))
K_policy_grid = np.zeros((A_grid_size, K_grid_size, T+1))

# Create array to store flow and total value over different next period state choices
flow_store = np.zeros((A_grid_size, K_grid_size))
V_total_store = np.zeros((A_grid_size, K_grid_size))

# Compute flow utility from each state to next state
# Iterate over initial state
for t0, T0 in np.ndenumerate(T_grid):
    for a0, A0 in np.ndenumerate(A_grid):
        for k0, K0 in np.ndenumerate(K_grid):
            
            # Iterate over next state and compute flow utility for any given next state
            for a1, A1 in np.ndenumerate(A_grid):
                for k1, K1 in np.ndenumerate(K_grid):
                    state = (A0, K0, A1, K1)
                    check = feasible(state, params)

                    if check == True:
                        
                        cons = {'type': 'ineq',
                        'fun': constraints,
                        'args': (state, params),
                        'lb': np.array([0, 0, 0, 0, 0]),
                        'ub': np.array([0, 0, 1, 1, np.inf])
                               }
                        
                        results = minimize(utility, [0.5, 0.5, 0.5], method = 'SLSQP', constraints = cons, args = (params,))
                        c, l, x = results.x
                        if c > 0 and l > 0:
                            flow_store[a1, k1] = np.log(c) + alpha*np.log(l)
                        else:
                            flow_store[a1, k1] = -1e99
                        
                    else:
                        flow_store[a1, k1] = -1e99
                
            # Add computed flow utilities to discounted value function next period
            if T0 == T:
                V_total_store = flow_store + gamma*np.log(A0)
            else:
                V_total_store = flow_store + beta*V_next[:, :, T0+1]
                
            # Now find next period state that maximizes flow + discounted value function
            max_value = np.amax(V_total_store)
            ind = np.unravel_index(np.argmax(V_total_store), V_total_store.shape)
            
            # Put value into updated value function
            V_init[a0, k0, t0] = max_value
            
            # Put optimal state choice into policy array
            A_policy_grid[a0, k0, t0] = ind[0]
            K_policy_grid[a0, k0, t0] = ind[1]
            
# Compare V_init and V_next
V_diff = np.absolute(V_init - V_next)
err = np.amax(V_diff)

In [74]:
err

25.460190920127513

In [75]:
V_next

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       ...,

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1.

In [76]:
np.amax(V_init)

26.460190920127513

In [77]:
V_init

array([[[ 0.06147829,  0.06147829,  0.06147829, ...,  0.06147829,
          0.06147829, -0.92852171],
        [ 0.45399815,  0.45399815,  0.45399815, ...,  0.45399815,
          0.45399815, -0.53600185],
        [ 0.71040697,  0.71040697,  0.71040697, ...,  0.71040697,
          0.71040697, -0.27959303],
        ...,
        [ 1.4670552 ,  1.4670552 ,  1.4670552 , ...,  1.4670552 ,
          1.4670552 ,  0.4770552 ],
        [ 1.56794996,  1.56794996,  1.56794996, ...,  1.56794996,
          1.56794996,  0.57794996],
        [ 1.65959211,  1.65959211,  1.65959211, ...,  1.65959211,
          1.65959211,  0.66959211]],

       [[ 0.62721688,  0.62721688,  0.62721688, ...,  0.62721688,
          0.62721688, -0.17892073],
        [ 0.86163311,  0.86163311,  0.86163311, ...,  0.86163311,
          0.86163311,  0.0554955 ],
        [ 1.03390567,  1.03390567,  1.03390567, ...,  1.03390567,
          1.03390567,  0.22776806],
        ...,
        [ 1.62649724,  1.62649724,  1.62649724, ...,  